In [1]:
import pandas as pd
from pulp import *

In [4]:
inputs = pd.read_csv('inputs.csv', index_col=0, delimiter=';')
outputs = pd.read_csv('outputs.csv', index_col=0, delimiter=';')


In [7]:
efficiency_scores = {}

for DMUo in inputs.index:
    problem = LpProblem("AirportEfficiency", LpMaximize)

    lambda_vars = LpVariable.dicts("Lambda", [str(idx) for idx in inputs.index], lowBound=0)

    problem += lpSum([lambda_vars[i] * outputs.loc[i, 'o1'] for i in outputs.index])

    for col in inputs.columns:
        problem += lpSum([lambda_vars[i] * inputs.loc[i, col] for i in inputs.index]) <= inputs.loc[DMUo, col]

    problem.solve()

    efficiency_score = value(problem.objective) / outputs.loc[DMUo, 'o1']
    efficiency_scores[DMUo] = efficiency_score

    print(f"Efektywność dla {DMUo}: {efficiency_score:.2f}")
    print("Jednostki referencyjne i ich wartości lambda:")
    for v in problem.variables():
        if v.varValue > 0:
            print(f"{v.name} = {v.varValue:.2f}")
    print("\n" + "="*50 + "\n")

efficiency_df = pd.DataFrame.from_dict(efficiency_scores, orient='index', columns=['Efficiency'])
efficiency_df.to_csv('efficiency_scores.csv')
print("Wyniki zapisane do pliku 'efficiency_scores.csv'.")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/joanna/miniconda3/envs/iswd/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/05/htkmyw9s0mj9r1tdvqjv8cg00000gn/T/3ff77bef820d489280f365e82e37aa51-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/05/htkmyw9s0mj9r1tdvqjv8cg00000gn/T/3ff77bef820d489280f365e82e37aa51-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 65 RHS
At line 70 BOUNDS
At line 71 ENDATA
Problem MODEL has 4 rows, 11 columns and 44 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (0) rows, 11 (0) columns and 44 (0) elements
0  Obj -0 Dual inf 18.451191 (11)
0  Obj -0 Dual inf 18.451191 (11)
2  Obj 9.5
Optimal - objective value 9.5
Optimal objective 9.5 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.